In [23]:
# install & import libraries
import google.generativeai as genai
import json
import time
import random
import re

from google.api_core.exceptions import TooManyRequests
from google.colab import userdata

# Get your API key from Colab secrets
GOOGLE_API_KEY = userdata.get('api_new')

genai.configure(api_key=GOOGLE_API_KEY)

# Define models
thinking_model = genai.GenerativeModel('gemini-2.5-pro')         # big brain model
lite_model      = genai.GenerativeModel('gemini-2.5-flash-lite') # model we optimize


In [24]:
def call_with_retry(func, max_retries=5):
    for attempt in range(max_retries):
        try:
            return func()
        except TooManyRequests as e:
            retry_after = getattr(e, "retry_after", None)
            if retry_after is None:
                retry_after = 2 ** attempt + random.random()
            print(f"Quota hit, retrying in {retry_after:.2f} seconds...")
            time.sleep(retry_after)
    raise Exception("Max retries exceeded")

In [25]:
def generate_with_lite(prompt: str) -> str:
    def _call():
        response = lite_model.generate_content(prompt)
        return getattr(response, "text", str(response))
    return call_with_retry(_call)


In [26]:
def get_feedback_and_optimized_prompt(lite_prompt: str, lite_output: str) -> dict:

    optimization_prompt = f"""
You are a prompt optimizer for a small text-generation model ("lite model").

Your tasks:
- Evaluate the original prompt + lite model output.
- Identify weaknesses.
- Suggest an improved prompt (same goal, but clearer, structured, more helpful).
- Explain your feedback.

Return ONLY valid JSON:
{{
  "optimized_prompt": "...",
  "feedback_text": "...",
  "rationale": "..."
}}

ORIGINAL_PROMPT:
{lite_prompt}

LITE_MODEL_OUTPUT:
{lite_output}
"""

    def _call():
        response = thinking_model.generate_content(optimization_prompt)
        return getattr(response, "text", str(response))

    raw_text = call_with_retry(_call).strip()

    # extract JSON in case model wrapped it in ```json ... ```
    match = re.search(r"```json\s*([\s\S]*?)\s*```", raw_text)
    json_str = match.group(1).strip() if match else raw_text

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Could not parse JSON. Keeping original prompt.")
        print("Raw thinking model output:\n", raw_text)
        return {
            "optimized_prompt": lite_prompt,
            "feedback_text": "Thinking model did not output valid JSON.",
            "rationale": "Prompt unchanged due to JSON parsing failure."
        }

    return {
        "optimized_prompt": data.get("optimized_prompt", lite_prompt),
        "feedback_text": data.get("feedback_text", ""),
        "rationale": data.get("rationale", ""),
    }


In [27]:
def run_optimization_loop(initial_prompt: str, num_turns: int = 5):

    current_prompt = initial_prompt
    print("=== START PROMPT OPTIMIZATION ===\n")

    for turn in range(1, num_turns + 1):
        print(f"\n--- TURN {turn} ---")

        print("\nCurrent prompt:")
        print("----------------------------------")
        print(current_prompt)

        print("\n[1] Calling lite model...")
        lite_output = generate_with_lite(current_prompt)
        print("----------------------------------")
        print(lite_output)

        print("\n[2] Getting improved prompt...")
        feedback = get_feedback_and_optimized_prompt(current_prompt, lite_output)

        print("\nFeedback:")
        print("----------------------------------")
        print(feedback["feedback_text"])

        print("\nRationale:")
        print("----------------------------------")
        print(feedback["rationale"])

        print("\nNew optimized prompt:")
        print("----------------------------------")
        print(feedback["optimized_prompt"])

        current_prompt = feedback["optimized_prompt"]

    print("\n=== END ===")
    print("\nFinal optimized prompt:")
    print("----------------------------------")
    print(current_prompt)


In [28]:
initial_prompt = "Napiši uvjerljiv tekst za oglas koji prodaje premium dizajnersku torbu."
num_iterations = 5

run_optimization_loop(initial_prompt, num_iterations)

=== START PROMPT OPTIMIZATION ===


--- TURN 1 ---

Current prompt:
----------------------------------
Napiši uvjerljiv tekst za oglas koji prodaje premium dizajnersku torbu.

[1] Calling lite model...
----------------------------------
Naravno, evo prijedloga za uvjerljiv oglas za premium dizajnersku torbu:

---

**Naslov:** **Više od torbe. Izjava. Vaša jedinstvena priča.**

**Glavni tekst:**

U svijetu koji žuri, gdje se svakodnevica često pretvara u monotoniju, postoji jedan predmet koji odskače. Predmet koji nosi priču, odražava Vaš stil i naglašava Vašu neusporedivu osobnost. Predstavljamo Vam [Ime Dizajnerske Torbe/Kolekcije], remek-djelo dizajna i umijeća, kreirano za one koji cijene savršenstvo u svakom detalju.

Ova torba nije samo modni dodatak; to je produžetak Vas. Izrađena od **najfinije [navedite vrstu kože, npr. talijanske teleće kože, egzotične kože]**, s nepogrešivom pažnjom prema svakom šavu i obliku, ona je sinonim za luksuz koji se osjeti. Mekana na dodir, a istovr

Quota hit, retrying in 1.58 seconds...


Quota hit, retrying in 2.74 seconds...


Quota hit, retrying in 4.91 seconds...


Quota hit, retrying in 8.17 seconds...


Quota hit, retrying in 16.54 seconds...


Exception: Max retries exceeded